In [3]:
import torch
from torch.autograd import Variable
import torch.utils
import torch.utils.data
from tqdm import tqdm

ImportError: No module named tqdm

In [ ]:
class LSTMTagger(nn.Module):

    def __init__(self, input_dim, hidden_dim, tagset_size, batch_size):
        super(LSTMTagger, self).__init__()

        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.tagset_size = tagset_size
        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(input_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden(self.batch_size)

    def init_hidden(self, minibatch_size):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (autograd.Variable(torch.zeros(1, minibatch_size, self.hidden_dim)),
                autograd.Variable(torch.zeros(1, minibatch_size, self.hidden_dim)))

    def forward(self, sentence_inp):
        if len(sentence_inp.size())==3:
            num_sentences = self.batch_size
        else:
            num_sentences = 1
        self.hidden = self.init_hidden(num_sentences)
        sentence = sentence_inp.view((155, num_sentences, -1))
        lstm_out, self.hidden = self.lstm(sentence, self.hidden)
        tag_space = self.hidden2tag(lstm_out)
        tag_scores = F.log_softmax(tag_space)
        tag_scores_out = tag_scores.view((num_sentences, 155, self.tagset_size))
        return tag_scores_out

In [ ]:
def run_epoch(dataset, is_training, model, optimizer, batch_size):
    data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                              shuffle=True, drop_last=True)
    losses = []
    if is_training:
        model.train()
    else:
        model.eval()

    total = 0.
    right = 0
    for batch in tqdm(data_loader):
        q_body = Variable(batch["q_body"]) # batch_size x 1 x truncate_length x 200]
        p_body = Variable(batch["p_body"])
        neg_bodies = Variable(batch["neg_bodies"]) # batch_size x num_negs x truncate_length x 200
        q_title = Variable(batch["q_title"])
        p_title = Variable(batch["p_title"])
        neg_titles = Variable(batch["neg_titles"])
        num_negs = neg_titles.size()[1]
        if is_training:
            optimizer.zero_grad()
        q_body_enc, q_title_enc = model(q_body), model(q_title) # batch_size x 1 x enc_length
        p_body_enc, p_title_enc = model(p_body), model(p_title)
        neg_body_encs, neg_title_encs = model(neg_bodies), model(neg_titles) # batch_size x num_negs x enc_length
        q_enc = q_title_enc + q_body_enc / 2.0
        p_enc = p_title_enc + p_body_enc / 2.0
        neg_encs = neg_title_encs + neg_body_encs / 2.0
        candidate_encs = torch.cat((p_enc, neg_encs), dim=1) #batch_size x (num_negs + 1) x enc_length
        query_encs = q_enc.repeat(1, num_negs, 1) # batch_size x (num_negs + 1) x enc_length
        cos = torch.nn.CosineSimilarity(dim=2, eps=1e-08)(candidate_encs, query_encs) # batch_size x (num_negs + 1)
        target = Variable(torch.zeros(batch_size).long(), requires_grad=True)
        loss = torch.nn.MultiMarginLoss()(cos, target)

        if is_training:
            loss.backward(retain_graph=True)
            optimizer.step()
        losses.append(loss.cpu().data[0])
    avg_loss = numpy.mean(losses)
    return avg_loss

In [ ]:
def train_model(train_data, model, batch_size=200, num_epochs=50, lr=1.0, weight_decay=0):
    print("start train_model")
    #optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    optimizer = torch.optim.Adam(model.parameters())
    for epoch in range(1, num_epochs + 1):
        print("epoch", epoch)
        train_loss, train_acc = run_epoch(train_data, True, model, optimizer, batch_size)
        print("train_loss", train_loss)
        print("train_acc", train_acc)

In [ ]:
import read_input
from read_input import QuestionDataset

In [ ]:
TEXT_TOKENIZED_FILE = 'askubuntu/text_tokenized.txt'
TRAIN_FILE = 'askubuntu/train_random.txt'
DEV_FILE = 'askubuntu/dev.txt'
TEST_FILE = 'askubuntu/test.txt'

TRUNCATE_LENGTH = 150

BATCH_SIZE = 200
NUM_EPOCHS = 50

train_dataset = QuestionDataset(TEXT_TOKENIZED_FILE, TRAIN_FILE, truncate=TRUNCATE_LENGTH)

train_model(train_dataset, model, num_epochs=NUM_EPOCHS, batch_size=BATCH_SIZE)